<a href="https://colab.research.google.com/github/nyoons/ESAA/blob/main/2022_11_18_%EA%B3%BC%EC%A0%9C_%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD%EA%B0%80%EC%9D%B4%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> **08. 분류 실습 - 캐글 산탄데르 고객 만족 예측**

370개의 피처. 이름만으로는 어떤 속성인지 추정 불가. 클래스 레이블 명은 TARGET이며 이 값이 1이면 불만, 0이면 만족. 성능은 ROC-AUC로 평가. 대부분이 만족이고 불만족은 일부라서 정확도보다는 ROC-AUC가 적합.

**데이터 전처리**

XGBoost는 사이킷런 래퍼를 이용.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

cust_df=pd.read_csv('train_santander.csv', encoding='latin-1')
print('dataset shape:', cust_df.shape)
cust_df.head(3)

dataset shape: (76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


371개의 피처 존재. 피처 타입과 null 값 알아보자.

In [ ]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


111개의 피처가 float형, 260개의 피처가 int형으로 모든 피처가 숫자 형이고 null은 없다.

전체 데이터에서 만족과 불만족 비율을 보자. target 속성의 값의 분포를 보면 됨.

In [ ]:
print(cust_df['TARGET'].value_counts())
unsatisfied_cnt=cust_df[cust_df['TARGET']==1].TARGET.count()
total_cnt=cust_df.TARGET.count()
print('unsatisfied 비율은 {0:.2f}'.format((unsatisfied_cnt/total_cnt)))

0    73012
1     3008
Name: TARGET, dtype: int64
unsatisfied 비율은 0.04


각 피처의 값 분포를 간단히 확인해보자.

In [ ]:
cust_df.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


var3 칼럼에서 min 이상. 꽤 많음. 가장 값이 많은 2로 변환하자.
<br/>ID는 드롭하는게 낫다.
<br/>클래스 데이터 세트와 피처 데이터 세트 분리해 별도의 데이터 세트로 저장하자.

In [ ]:
cust_df['var3'].replace(-999999,2,inplace=True)
cust_df.drop('ID', axis=1, inplace=True)

#피처 세트와 레이블 세트 분리, 레이블 칼럼은 DF의 맨 마지막에 위치해 칼럼 위치 -1로 분리.
X_features=cust_df.iloc[:, :-1]
y_labels=cust_df.iloc[:,-1]
print('피처 데이터 shape:{0}'.format(X_features.shape))

피처 데이터 shape:(76020, 369)


학습/테스트 분리하자. 비대칭한 데이터 세트이므로 클래스인 Target 값 분포가 학습/테스트 데이터에 모두 비슷하게 추출되었는지 확인 필요.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X_features, y_labels, test_size=0.2, random_state=0)
train_cnt=y_train.count()
test_cnt=y_test.count()
print('학습 세트 shape:{0}, 테스트 세트 shape:{1}'.format(X_train.shape, X_test.shape))

print('학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n 테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_cnt)

학습 세트 shape:(60816, 369), 테스트 세트 shape:(15204, 369)
학습 세트 레이블 값 분포 비율
0    0.960964
1    0.039036
Name: TARGET, dtype: float64

 테스트 세트 레이블 값 분포 비율
0    0.9583
1    0.0417
Name: TARGET, dtype: float64


대충 비슷.

**XGBoost 모델 학습과 하이퍼 파라미터 튜닝**

먼저 학습 모델 생성하고 결과를 평가해보자. 사이킷런 래퍼 XGBClassifier 기반으로 학습 수행.

In [ ]:
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

#n_estimators=500, random_state는 예제 수행 시마다 동일 예측 결과를 위해 설정.
xgb_clf=XGBClassifier(n_estimators=500, random_state=156)

#성능 평가 지표를 auc로, 조기 중단 파라미터는 100으로 설정하고 학습 수행.
xgb_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_train, y_train), (X_test, y_test)])

xgb_roc_score=roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.799928	validation_1-auc:0.803548
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.802222	validation_1-auc:0.805222
[2]	validation_0-auc:0.80819	validation_1-auc:0.813162
[3]	validation_0-auc:0.8127	validation_1-auc:0.813243
[4]	validation_0-auc:0.81648	validation_1-auc:0.816979
[5]	validation_0-auc:0.816018	validation_1-auc:0.816629
[6]	validation_0-auc:0.816474	validation_1-auc:0.817776
[7]	validation_0-auc:0.818148	validation_1-auc:0.818464
[8]	validation_0-auc:0.81806	validation_1-auc:0.818295
[9]	validation_0-auc:0.817039	validation_1-auc:0.818087
[10]	validation_0-auc:0.818318	validation_1-auc:0.818749
[11]	validation_0-auc:0.818711	validation_1-auc:0.818521
[12]	validation_0-auc:0.818673	validation_1-auc:0.818516
[13]	validation_0-auc:0.819156	validation_1-auc:0.818998
[14]	validation_0-auc:0.819847	validation_1-auc:0.81999
[15

하이퍼 파라미터 튜닝 진행하자. 칼럼 수가 많으므로 과적합 가능성 가정하고, max_depth, min_child_weight, colsample_bytree 하이퍼 파라미터만 일차 튜밍 대상으로 한다.

다음 코드는 8개의 하이퍼 파라미터 경우의 수를 가짐. GridSearchCV 적용 후에 초적 파라미터가 어떤 것이고, GridSearchCV에서 재학습된 estimator에서 ROC-AUC 수치가 어떻게 향상되는지 확인하자.

In [ ]:
from sklearn.model_selection import GridSearchCV

#하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators 감소시킴
xgb_clf=XGBClassifier(n_estimators=100)

params={'max_depth':[5,7], 'min_child_weight':[1,3], 'colsample_bytree':[0.5, 0.75]}

#cv=3으로 지정
gridcv=GridSearchCV(xgb_clf, param_grid=params, cv=3)
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric='auc', eval_set=[(X_train, y_train), (X_test, y_test)])

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)

xgb_roc_score=roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

이전 예제의 roc 보다 colsample_bytree=0.75, max_depth=7, min_child_weight=1에서 조금 개선됨. 앞에서 구한 최적화 하이퍼 파라미터를 기반으로 다른 하이퍼 파라미터를 변경, 추가해 다시 최적화 진행하자.

In [ ]:
#n_estimators=1000으로 증가시키고, learning_rate=0.02로 감소, reg_alpha=0.03 추가.
xgb_clf=XGBClassifier(n_estimators=1000, random_state=156, learning_rate=0.02, max_depth=7, min_child_weight=1, colsample_bytree=0.75, reg_alpha=0.03)

#성능 평가 지표를 auc로, 조기 중단 파라미터 값을 200으로 설정하고 학습 수행
xgb_clf.fit(X_train, y_train, early_stopping_rounds=200, eval_metric='auc', eval_set=[(X_train, y_train), (X_test, y_test)])

xgb_roc_score=roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

확실히 이전보다 나은 결과. 아쉬운 점은 시간. 사실 앙상블 계열 알고리즘에서 하이퍼 파라미터 튜닝으로 성능 수치가 급격히 개선되는 경우는 잘 없음.

피처 중요도를 그래프로 나타내보자.

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax=plt.subplots(1,1, figsize=(10,8))
plot_importance(xgb_clf, ax=ax, max_num_features=20, height=0.4)

var38, var15가 예측 성능을 좌우함.

**LightGBM 모델 학습과 하이퍼 파라미터 튜닝**

앞 예제에서 만들어진 데이터세트 기반으로 수행하고 측정할 것.

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf=LGBMClassifier(n_estimators=500)

evals=[(X_test, y_test)]
lgbm_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='auc', eval_set=evals, verbose=True)

lgbm_roc_score=roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC : {0:.4f}'.format(lgbm_roc_score))

XGBoost보다 확실히 시간은 덜 걸림. GridSearcCV로 다양한 하이퍼 파라미터 튜닝해보자.

In [ ]:
from sklearn.model_selection import GridSearchCV

#하이퍼 파라미터 테스트의 수행 속도 향상을 위해 n_estimators 감소시킴
lgbm_clf=LGBMClassifier(n_estimators=200)

params={'num_leaves':[32,64], 'max_depth':[128, 160], 'min_child_samples':[60, 100], 'sub_sample':[0.8,1]}

#cv=3
gridcv=GridSearchCV(lgbm_clf, param_grid=params, cv=3)
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric='auc', eval_set=[(X_train, y_train), (X_test, y_test)])

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)
lgbm_roc_score=roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

해당 파라미터를 다시 적용해 결과를 도출해보자.

In [ ]:
lgbm_clf=LGBMClassifier(n_estimators=1000, num_leaves=32, subsample=0.8, min_child_samples=100, max_depth=128)
evals=[(X_test, y_test)]
lgbm_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='auc', eval_set=evals, verbose=True)

lgbm_roc_score=roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC : {0.4f}'.format(lgbm_roc_score))

lgbm의 경우 0.8442로 측정됨.

>**09. 분류 실습 - 캐글 신용카드 사기 검출**

매우 불균형한 분포. class의 0은 정상적인 데이터 1은 사기 트랜잭션 의미. 전체 데이터의 0.172%만이 사기 트랜잭션임. 


**언더 샘플링과 오버 샘플링의 이해**

불균형할 경우 예측 성능에 문제 발생 가능성. 적절한 학습 데이터를 확보하는 방안이 필요한데 대표적을 오버 샘플링과 언더 샘플링 방식이 있음.

* 언더 샘플링 : 많은 데이터 세트를 적은 데이터 세트 수준으로 감소시키는 방식. 하지만 너무 많은 정상 레이블 데이터를 감소시켜 오히려 제대로 학습 불가능 할 수 있음.
* 오버 샘플링 : 적은 데이터를 증식하여 학습을 위한 충분한 데이터를 확보하는 방식. 단순 증식은 오버비팅 되기 때문에 원본 피처 값을 아주 약간만 변형하여 증식한다. SMOTE 방법.

SMOTE 방법의 대표적 패키지는 imbalanced-learn. 증식해보자.

**데이터 일차 가공 및 모델 학습/예측/평가**


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filtetwarnings('ignore')
%matplotlib inline

card_df=pd.read_csv('creditcard.csv')
card_df.head(3)

v로 시작하는 피처들의 의미 모름. time은 제거하자. 다양한 가공 수행하고 예측 성능 비교하자.

In [ ]:
from sklearn.model_selection import train_test_split

#인자로 입력받은 dF를 복사하고 time 칼럼 삭제하고 복사된 DF 반환
def get_preprocessed_df(df=None):
  df_copy=df.copy()
  df_copy.drop('Time', axis=1, inplace=True)
  return df_copy

In [ ]:
#사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수
def get_train_test_dataset(df=None):
  #인자로 입력된 DF의 사전 데이터 가공이 완료된 복사DF반환
  df_copy=get_preprocessed_df(df)
  #DF의 맨 마지막 칼럼이 레이블, 나머지는 피처들
  X_features=df_copy.iloc[:,:-1]
  y_target=df_copy.iloc[:,-1]
  #train_test_split으로 학습과 테스트 데이터 분할. stratiy=y_target으로 stratified 기반 분할
  X_train, X_test, y_train, y_test=train_test_split(X_features, y_target, test_size=0.3, random_state=0, stratify=y_target)
  #학습/테스트 데이터 세트반환
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test=get_train_test_dataset(card_df)

서로 비슷하게 분할되었는지 확인

In [ ]:
print('학습 데이터 레이블 값 비율')
print(y_train.value_counts()/y_train.shape[0]*100)
print('테스트 데이터 레이블 값 비율')
print(y_test.value_counts()/y_test.shape[0]*100)

잘 분할됨. 이제 모델 만들자. 로지스틱 회귀와 lgbm 기반의 모델이 데이터 가공을 수행하며 예측 성능이 어떻게 변하는지 살펴볼 것.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf=LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_pred=lr_clf.predict(X_test)
lr_pred_proba=lr_clf.predict_proba(X_test)[:,1]

#3장에서 사용한 get_clf_eval 함수 이용해 평가 수행
get_clf_eval(y_test, lr_pred, lr_pred_proba)

이때는 재현율=0.6081, roc=0.9709

이제는 lgbm으로 모델 만들어보자.

In [ ]:
#인자로 사이킷런의 estimator 객체와 학습/테스트 데이터 세트 입력 받아 학습/예측/평가 수행
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
  model.fit(ftr_train, tgt_train)
  pred=model.predict(ftr_test)
  pred_proba=model.predict_proba(ftr_test)[:,1]
  get_clf_eval(tgt_test, pred, pred_proba)

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf=LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

재현율=0.7568, roc=0.9797로 앞의 로지스틱보단 낫다.

**데이터 분포도 변환 후 모델 학습/예측/평가**

왜곡된 분포도를 가지는 데이터를 재가공한 뒤에 모델을 다시 테스트해보자. 먼저 중요 피처 값의 분포를 살펴보자.

In [ ]:
import seabon as sns
plt.figure(figsize=(8,4))
plt.xticks(range(0, 30000, 1000), rotation=60)
sns.displot(card_df['Amount'])

amount 피처는 사용금액이 1000불 이하인 데이터가 대부분임. 하지만 꼬리가 길다. 이를 표준 정규 분포 형태로 변환한 뒤에 로지스틱 회귀의 예측 성능을 보자.

In [ ]:
from sklearn.preprocessing import StandardScaler
#사이킷런의 StandardScaler를 이용해 정규 분포 형태로 Amount 피처 값 변환하는 로직 수정.
def get_preprocessed_df(df=None):
  df_copy=df.copy()
  scaler=StandardScaler()
  amount_n=scaler.fit_transform(df_copy['Amount'].values.reshape(-1,1))
  #변환된 amount를 amount_scaled로 피처명 변경 후 DF 맨 앞 칼럼으로 입력
  df_copy.insert(0, 'Amount_Scaled', amount_n)
  #기존 time, amount 피처 삭제
  df_copy.drop(['Time', 'Amount'], axis=1, inplace=True)
  return df_copy

로지스틱 회귀와 lgbm 모델을 각각 학습/평가/예측 해보자.

In [ ]:
#amount를 정규 분포 형태로 변환 후 로지스틱 회귀 및 lgbm 수행.
X_train, X_test, y_train, y_test=get_train_test_dataset(card_df)

print('###로지스틱 회귀 예측 성능###')
lr_clf=LogisticRegression()
get_model_train_eval(lr_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train,tgt_test=y_test)

print('###lgbm 예측 성능###')
lgbm_clf=LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1)
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

성능이 크게 개선되지는 않는다. 

이번에는 로그 변환을 수행해보자. 분포가 심하게 왜곡되어 있을 경우 유용.

In [ ]:
def get_preprocessed_df(df=None):
  df_copy=df.copy()
  #넘파이의 log1p()이용해 amount를 로그 변환
  amount_n=np.log1p(df_copy['Amount'])
  df_copy.insert(0, 'Amount_scaled', amount_n)
  df_copy.drop(['Time', 'Amount'], axis=1, inplace=True)
  return df_copy

In [ ]:
X_train, X_test, y_train, y_test=get_train_test_dataset(card_df)

print('###로지스틱 회귀 예측 성능###')
get_model_train_eval(lr_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

print('###lgbm 예측 성능###')
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

모두 개선됨!

**이상치 데이터 제거 후 모델 학습/예측/평가**

IQR 이용해 제거하면 된다. 어떤 피처의 이상치를 검출할 것인지 선택이 필요. 결정값과 상관이 높은 피처들 위주로 이상치를 검출하는 것이 좋다.

In [ ]:
import seaborn as sns

plt.figure(figsize=(9,9))
corr=card_df.corr()
sns.heatmap(corr, cmap='RdBu')

상관관계를 보니 v14, v17이 상관관계가 높다. v14의 이상치를 제거하자.

In [ ]:
import numpy as np

def get_outlier(df=None, column=None, weight=1.5):
  #fraud에 해당하는 칼럼 데이터만 추출, Q1, Q3 지점을 np.percentile로 추출
  fraud=df[df['Class']==1][column]
  quantile_25=np.percentile(fraud.values, 25)
  quantile_75=np.percentile(fraud.values, 75)
  #IQR 구하고 1.5 곱해 지점 구함
  iqr=quantile_75-quantile_25
  iqr_weight=iqr*weight
  lowest_val=quantile_25-iqr_weight
  highest_val-quantile_75+iqr_weight
  #범위 밖 값을 이상치로 설정하고 DF index 반환
  outlier_index=fraud[(fraud<lowest_val)|(fraud>highest_val)].index
  return outlier_index

In [ ]:
#찾자
outlier_index=get_outlier(df=card_df, cloumn='V14', weight=1.5)
print('이상치 데이터 인덱스:', outlier_index)

4개의 이상치. 삭제하고 가공한 뒤 다시 로지스틱 회귀와 lgbm 적용하면 된다. 성능이 크게 향상된다. 

**SMOTE 오버 샘플링 적용 후 모델 학습/예측/평가**

반드시 학습 데이터 세트만 오버 샘플링을 해야 한다.

In [ ]:
from imblearn.over_sampling import SMOTE

smote=SMOTE(random_state=0)
X_train_over, y_train_over=smote.fit_sample(X_train, y_train)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트:', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트:', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

데이터 증식됨! 로지스틱 학습 시키고 평가해보자.

In [ ]:
lr_clf=LogisticRegression()
#ftr_train과 tgt_train 인자값이 sMOTE 증식된 값으로 받아야함.
get_model_train_eval(lr_clf, ftr_train=X_train_over, ftr_test=X_test, tgt_train=y_train_over, tgt_test=y_test)

재현율이 크게 증가하나 정밀도가 급격하게 저하됨. 오버 샘플링으로 인한 문제. 어떤 문제가 발생하고 있는 것인가?

In [ ]:
precision_recall_curve_plot(y_test, lr_clf.predict_proba(X_test)[:,1])

임계값 0.99 이하에서는 재현율이 매우 좋고 정밀도가 극단적으로 낮다가 0.99 이상에서는 정반대로... 분류 결정 임계값 조정하더라도 민감도 너무 심해 올바른 재현율/정밀도 성능 얻을 수 없으므로 로지스틱 회귀 모델의 경우 SMOTE가 적당해보이지 않는다. 그럼 lgbm을 확인해보자.

In [ ]:
lgbm_clf=LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
get_model_train_eval(lgbm_clf, ftr_train=X_train_over, ftr_test=X_test, tgt_train=y_train_over, tgt_test=y_test)

재현율이 이상치만 제거했을 경우보다 나아졌다. 하지만 정밀도는 이전보다 낮음. SMOTE를 적용하면 이렇게 재현율을 높아지나 정밀도는 낮아지는 것이 일반적이다. 